In [132]:
import skmultilearn
from skmultilearn import dataset
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, hamming_loss, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from skmultilearn.adapt import MLkNN

from deap import base, creator, gp, tools, algorithms
from itertools import combinations
import operator, math, random
import numpy as np
import pygraphviz as pgv
import sympy

In [133]:
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)

pset.renameArguments(ARG0='x')

In [134]:
e_train = skmultilearn.dataset.load_dataset('emotions', 'train')
e_test = skmultilearn.dataset.load_dataset('emotions', 'test')

emotions:train - exists, not redownloading
emotions:test - exists, not redownloading


In [135]:
# X, y, names of attributes, names of labels
X_train = e_train[0].toarray() # 391, 72
y_train = e_train[1].toarray() # 391, 6

In [136]:
y_train_u = np.unique(y_train, axis=0) # 26 unique labels

In [137]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def to_dec(label):
    # label to integer
    str_list = list(str(x) for x in label)
    str_bin = ''.join(str_list)
    return int(str_bin, 2)

def evalSymbReg(individual, labels):
    # transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    loss = 0
    
    for i in range(len(labels)):
        label_l = []
        for j in range(len(labels)):
            if i != j:
                label_l.append([np.count_nonzero(labels[i] != labels[j]),
                                func(to_dec(labels[j]))])
        
        closest_orig = sorted(label_l)[:5] # closest original labels
        closest_new = sorted(label_l, key=lambda x: x[1])[:5] # closest new labels
        
        print(closest_orig)
        
        for l in closest_orig:
            if l not in closest_new:
                loss += 1
        
    return loss,

toolbox.register("evaluate", evalSymbReg, labels=y_train_u)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

def main():
    random.seed(0)

    pop = toolbox.population(n=50)
    hof = tools.HallOfFame(1)

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", np.mean)
    mstats.register("std", np.std)
    mstats.register("min", np.min)
    mstats.register("max", np.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 20, stats=mstats,
                                   halloffame=hof, verbose=True) # P(mate), P(mutate), ngen
    # print log
    return pop, log, hof # returns population, log, hall of fame

progs = main()

[[1, 18], [1, 162], [1, 578], [1, 2178], [2, 8]]
[[1, 18], [1, 72], [1, 200], [1, 648], [1, 2312]]
[[1, 2], [1, 8], [1, 98], [1, 242], [2, 72]]
[[1, 72], [1, 288], [1, 800], [2, 2], [2, 8]]
[[1, 8], [1, 32], [1, 98], [1, 392], [2, 18]]
[[1, 18], [1, 72], [2, 2], [2, 8], [2, 32]]
[[1, 162], [1, 200], [1, 288], [1, 1152], [1, 3200]]
[[1, 2], [1, 128], [1, 242], [1, 338], [2, 18]]
[[1, 8], [1, 128], [1, 242], [1, 392], [2, 18]]
[[1, 18], [1, 162], [1, 200], [2, 2], [2, 8]]
[[1, 32], [1, 128], [1, 338], [1, 392], [1, 1568]]
[[1, 162], [1, 288], [2, 2], [2, 32], [2, 98]]
[[1, 72], [1, 200], [1, 288], [2, 8], [2, 32]]
[[1, 578], [1, 648], [1, 800], [1, 1152], [1, 4608]]
[[1, 2], [1, 512], [1, 4802], [2, 18], [2, 162]]
[[1, 8], [1, 512], [2, 18], [2, 72], [2, 200]]
[[1, 32], [1, 512], [1, 1568], [2, 72], [2, 288]]
[[1, 128], [1, 512], [1, 1568], [1, 6272], [2, 162]]
[[1, 288], [1, 800], [1, 1152], [2, 32], [2, 128]]
[[1, 2178], [1, 2312], [1, 3200], [1, 4608], [2, 2]]
[[1, 2], [1, 2048], [1, 

[[1, 9], [1, 36], [2, 1], [2, 4], [2, 16]]
[[1, 81], [1, 100], [1, 144], [1, 576], [1, 1600]]
[[1, 1], [1, 64], [1, 121], [1, 169], [2, 9]]
[[1, 4], [1, 64], [1, 121], [1, 196], [2, 9]]
[[1, 9], [1, 81], [1, 100], [2, 1], [2, 4]]
[[1, 16], [1, 64], [1, 169], [1, 196], [1, 784]]
[[1, 81], [1, 144], [2, 1], [2, 16], [2, 49]]
[[1, 36], [1, 100], [1, 144], [2, 4], [2, 16]]
[[1, 289], [1, 324], [1, 400], [1, 576], [1, 2304]]
[[1, 1], [1, 256], [1, 2401], [2, 9], [2, 81]]
[[1, 4], [1, 256], [2, 9], [2, 36], [2, 100]]
[[1, 16], [1, 256], [1, 784], [2, 36], [2, 144]]
[[1, 64], [1, 256], [1, 784], [1, 3136], [2, 81]]
[[1, 144], [1, 400], [1, 576], [2, 16], [2, 64]]
[[1, 1089], [1, 1156], [1, 1600], [1, 2304], [2, 1]]
[[1, 1], [1, 1024], [1, 2401], [2, 9], [2, 81]]
[[1, 4], [1, 1024], [2, 9], [2, 36], [2, 100]]
[[1, 64], [1, 1024], [1, 3136], [2, 81], [2, 100]]
[[1, 256], [1, 1024], [1, 2401], [1, 3136], [2, 289]]
[[1, 289], [1, 1089], [1, 2304], [2, 1], [2, 256]]
[[1, 576], [1, 1600], [1, 2304]

[[1, 1.0], [1, 1.0], [1, 1.0], [1, 1.0], [2, 1.0]]
[[1, 1.0], [1, 1.0], [1, 1.0], [2, 1.0], [2, 1.0]]
[[1, 1.0], [1, 1.0], [1, 1.0], [1, 1.0], [2, 1.0]]
[[1, 1.0], [1, 1.0], [1, 1.0], [2, 1.0], [2, 1.0]]
[[1, 1.0], [1, 1.0], [2, 1.0], [2, 1.0], [2, 1.0]]
[[1, 1.0], [1, 1.0], [1, 1.0], [2, 1.0], [2, 1.0]]
[[1, 1.0], [1, 1.0], [1, 1.0], [1, 1.0], [2, 1.0]]
[[1, 1.0], [1, 1.0], [1, 1.0], [2, 1.0], [2, 1.0]]
[[1, 1.0], [1, 1.0], [1, 1.0], [2, 1.0], [2, 1.0]]
[[1, 0.030303030303030304], [1, 0.058823529411764705], [1, 0.1111111111111111], [1, 0.3333333333333333], [2, 0.02040816326530612]]
[[1, 0.029411764705882353], [1, 0.05555555555555555], [1, 0.1], [1, 0.16666666666666666], [1, 0.3333333333333333]]
[[1, 0.09090909090909091], [1, 0.14285714285714285], [1, 0.5], [1, 1.0], [2, 0.029411764705882353]]
[[1, 0.05], [1, 0.08333333333333333], [1, 0.16666666666666666], [2, 0.03125], [2, 0.03571428571428571]]
[[1, 0.07142857142857142], [1, 0.14285714285714285], [1, 0.25], [1, 0.5], [2, 0.02941176470

[[1, 0], [1, 0], [1, 0], [1, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [2, 0]]
[[1, 0], [1, 0], [2, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]
[[1, 0], [1, 0], [2, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]
[[1, 0], [1, 0], [1, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [2, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [2, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [2, 0]]
[[1, 0], [1, 0],

[[1, -34], [1, -18], [1, -10], [1, -6], [1, -3]]
[[1, -11], [1, -7], [1, -2], [1, -1], [2, -34]]
[[1, -20], [1, -12], [1, -6], [2, -32], [2, -28]]
[[1, -14], [1, -7], [1, -4], [1, -2], [2, -34]]
[[1, -6], [1, -3], [2, -14], [2, -13], [2, -11]]
[[1, -40], [1, -24], [1, -12], [1, -10], [1, -9]]
[[1, -13], [1, -11], [1, -8], [1, -1], [2, -40]]
[[1, -14], [1, -11], [1, -8], [1, -2], [2, -40]]
[[1, -10], [1, -9], [1, -3], [2, -14], [2, -13]]
[[1, -28], [1, -14], [1, -13], [1, -8], [1, -4]]
[[1, -12], [1, -9], [2, -28], [2, -14], [2, -11]]
[[1, -12], [1, -10], [1, -6], [2, -28], [2, -13]]
[[1, -48], [1, -24], [1, -20], [1, -18], [1, -17]]
[[1, -49], [1, -16], [1, -1], [2, -48], [2, -33]]
[[1, -16], [1, -2], [2, -48], [2, -34], [2, -24]]
[[1, -28], [1, -16], [1, -4], [2, -48], [2, -24]]
[[1, -56], [1, -28], [1, -16], [1, -8], [2, -48]]
[[1, -24], [1, -20], [1, -12], [2, -56], [2, -16]]
[[1, -48], [1, -40], [1, -34], [1, -33], [2, -56]]
[[1, -49], [1, -32], [1, -1], [2, -48], [2, -40]]
[[1, -3

...

[[1, 1.0], [1, 1.0], [1, 1.0], [2, 1.0], [2, 1.0]]
[[1, 1.0], [1, 1.0], [1, 1.0], [1, 1.0], [2, 1.0]]
[[1, 1.0], [1, 1.0], [1, 1.0], [2, 1.0], [2, 1.0]]
[[1, 1.0], [1, 1.0], [2, 1.0], [2, 1.0], [2, 1.0]]
[[1, 1.0], [1, 1.0], [1, 1.0], [2, 1.0], [2, 1.0]]
[[1, 1.0], [1, 1.0], [1, 1.0], [1, 1.0], [2, 1.0]]
[[1, 1.0], [1, 1.0], [1, 1.0], [2, 1.0], [2, 1.0]]
[[1, 1.0], [1, 1.0], [1, 1.0], [2, 1.0], [2, 1.0]]
[[1, -0.0], [1, -0.0], [1, -0.0], [1, -0.0], [2, -0.0]]
[[1, -0.0], [1, -0.0], [1, -0.0], [1, -0.0], [1, -0.0]]
[[1, -0.0], [1, -0.0], [1, -0.0], [1, -0.0], [2, -0.0]]
[[1, -0.0], [1, -0.0], [1, -0.0], [2, -0.0], [2, -0.0]]
[[1, -0.0], [1, -0.0], [1, -0.0], [1, -0.0], [2, -0.0]]
[[1, -0.0], [1, -0.0], [2, -0.0], [2, -0.0], [2, -0.0]]
[[1, -0.0], [1, -0.0], [1, -0.0], [1, -0.0], [1, -0.0]]
[[1, -0.0], [1, -0.0], [1, -0.0], [1, -0.0], [2, -0.0]]
[[1, -0.0], [1, -0.0], [1, -0.0], [1, -0.0], [2, -0.0]]
[[1, -0.0], [1, -0.0], [1, -0.0], [2, -0.0], [2, -0.0]]
[[1, -0.0], [1, -0.0], [1, -0.0]

[[1, 0.0], [1, 0.0], [1, 0.0], [1, 0.0], [1, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], [2, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [2, 0.0], [2, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], [2, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], [1, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], [2, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], [1, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], [2, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [2, 0.0], [2, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], [2, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], [1, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], [2, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], [2, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], [1, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], [1, 0.0], [1, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], [1, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], [2, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], [1, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [2, 0.0], [2, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], 

[[1, 1.0], [1, 1.0], [2, 1.0], [2, 1.0], [2, 1.0]]
[[1, 1.0], [1, 1.0], [1, 1.0], [2, 1.0], [2, 1.0]]
[[1, 1.0], [1, 1.0], [1, 1.0], [1, 1.0], [2, 1.0]]
[[1, 1.0], [1, 1.0], [1, 1.0], [2, 1.0], [2, 1.0]]
[[1, 1.0], [1, 1.0], [1, 1.0], [2, 1.0], [2, 1.0]]
[[1, 15.0], [1, 153.0], [1, 561.0], [1, 2145.0], [2, 6.0]]
[[1, 15.0], [1, 66.0], [1, 190.0], [1, 630.0], [1, 2278.0]]
[[1, 1.0], [1, 6.0], [1, 91.0], [1, 231.0], [2, 66.0]]
[[1, 66.0], [1, 276.0], [1, 780.0], [2, 1.0], [2, 6.0]]
[[1, 6.0], [1, 28.0], [1, 91.0], [1, 378.0], [2, 15.0]]
[[1, 15.0], [1, 66.0], [2, 1.0], [2, 6.0], [2, 28.0]]
[[1, 153.0], [1, 190.0], [1, 276.0], [1, 1128.0], [1, 3160.0]]
[[1, 1.0], [1, 120.0], [1, 231.0], [1, 325.0], [2, 15.0]]
[[1, 6.0], [1, 120.0], [1, 231.0], [1, 378.0], [2, 15.0]]
[[1, 15.0], [1, 153.0], [1, 190.0], [2, 1.0], [2, 6.0]]
[[1, 28.0], [1, 120.0], [1, 325.0], [1, 378.0], [1, 1540.0]]
[[1, 153.0], [1, 276.0], [2, 1.0], [2, 28.0], [2, 91.0]]
[[1, 66.0], [1, 190.0], [1, 276.0], [2, 6.0], [2, 28

[[1, 0.0], [1, 0.0], [2, 0.0], [2, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], [2, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], [1, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], [2, 0.0], [2, 0.0]]
[[1, 0.0], [1, 0.0], [1, 0.0], [2, 0.0], [2, 0.0]]
[[1, 3], [1, 9], [1, 17], [1, 33], [2, 2]]
[[1, 3], [1, 6], [1, 10], [1, 18], [1, 34]]
[[1, 1], [1, 2], [1, 7], [1, 11], [2, 6]]
[[1, 6], [1, 12], [1, 20], [2, 1], [2, 2]]
[[1, 2], [1, 4], [1, 7], [1, 14], [2, 3]]
[[1, 3], [1, 6], [2, 1], [2, 2], [2, 4]]
[[1, 9], [1, 10], [1, 12], [1, 24], [1, 40]]
[[1, 1], [1, 8], [1, 11], [1, 13], [2, 3]]
[[1, 2], [1, 8], [1, 11], [1, 14], [2, 3]]
[[1, 3], [1, 9], [1, 10], [2, 1], [2, 2]]
[[1, 4], [1, 8], [1, 13], [1, 14], [1, 28]]
[[1, 9], [1, 12], [2, 1], [2, 4], [2, 7]]
[[1, 6], [1, 10], [1, 12], [2, 2], [2, 4]]
[[1, 17], [1, 18], [1, 20], [1, 24], [1, 48]]
[[1, 1], [1, 16], [1, 49], [2, 3], [2, 9]]
[[1, 2], [1, 16], [2, 3], [2, 6], [2, 10]]
[[1, 4], [1, 16], [1, 28], [2, 6], [2, 12]]
[[1, 8], [1,

[[1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [2, 0]]
[[1, 0], [1, 0], [2, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]
[[1, 0], [1, 0], [2, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]
[[1, 0], [1, 0], [1, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [2, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [2, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [1, 0], [2, 0]]
[[1, 0], [1, 0], [1, 0], [2, 0], [2, 0]]
[[1, 0], [1, 0],

In [138]:
nodes, edges, labels = gp.graph(progs[2][0]) # hof

print(gp.PrimitiveTree(progs[2][0]))

g = pgv.AGraph()
g.add_nodes_from(nodes)
g.add_edges_from(edges)
g.layout(prog="dot")

for i in nodes:
    n = g.get_node(i)
    n.attr["label"] = labels[i]

g.draw("tree.pdf")

mul(mul(x, x), sub(x, x))
